In [127]:
# Trabajaremos a partir de las tablas del EDA que habíamos realizados para identificar cuales eran las averías más repetitivas y cuales eran las estaciones con mayor número de retrasos y aquellos
# donde se tarbada mas y menos en resolverse las averías.

# Identificamos que las bases donde se revolvían antes las averías eran las dos principales PENDIENTE DE PONER CUALES SON 
# El objetivo de este predictor de averías es identificar cuando podrían suceder las averías para que cuando esten más próximas a ocurridos posiconar los aviones con vuelos en las bases principales con personal
# de mantenimiento


#TRANSFORMACIÓN DE DATOS

In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import csv

In [129]:
# Histórico de averías
eventos_retrasos = pd.read_csv("./data/eventos_con_retrasos.csv", encoding= "latin1")
eventos_retrasos

,ID,Occurred,At,AC,Station,Headline,FlightNo,ATA,Minutos
0,12185,2021-01-02,18:59:00,AviÃ³n 1,PIQ,98254/1,NT421,77-00,38.0
1,12194,2021-01-03,20:15:00,AviÃ³n 2,PDQ,0097630/1,NT312,33-49,15.0
2,12200,2021-01-04,06:50:00,AviÃ³n 3,PIQ,MAINTENANCE ACTION WITHOUT PILOT REPORT - OPER...,414,25-65,3.0
3,12210,2021-01-04,14:13:00,AviÃ³n 3,PDQ,0096673/1,NT158,52-42,12.0
4,12217,2021-01-05,07:33:00,AviÃ³n 4,PIQ,OTHERS / NOT SPECIFIED - OPERATIONAL,NaN,33-00,8.0
...,...,...,...,...,...,...,...,...,...
4519,68953,2024-07-19,19:00:00,AviÃ³n 39,PDQ,2 ANOT: PARASOL CM1 Y ROLLER IZQ CM1,NT273,25-13,3.0
4520,68980,2024-07-20,06:25:00,AviÃ³n 4,PDQ,MTTO SIN INFO,NT201,52-31,5.0
4521,69000,2024-07-20,14:27:00,AviÃ³n 25,PIQ,SALTA AL ARRANCAR AVIONIC SINGLE SWITCH,NT141,42-11,30.0
4522,69007,2024-07-20,18:48:00,AviÃ³n 25,PDQ,TEMPERATURA ALTA DE COMBUSTIBLE ENG #1,NT178,73-15,40.0


In [130]:
# Modificamos el campo de AC ya que tras haber creado la tabla no reconoce la tilde
eventos_retrasos["AC"] = "avion " + eventos_retrasos["AC"].str[6:]
eventos_retrasos

,ID,Occurred,At,AC,Station,Headline,FlightNo,ATA,Minutos
0,12185,2021-01-02,18:59:00,avion 1,PIQ,98254/1,NT421,77-00,38.0
1,12194,2021-01-03,20:15:00,avion 2,PDQ,0097630/1,NT312,33-49,15.0
2,12200,2021-01-04,06:50:00,avion 3,PIQ,MAINTENANCE ACTION WITHOUT PILOT REPORT - OPER...,414,25-65,3.0
3,12210,2021-01-04,14:13:00,avion 3,PDQ,0096673/1,NT158,52-42,12.0
4,12217,2021-01-05,07:33:00,avion 4,PIQ,OTHERS / NOT SPECIFIED - OPERATIONAL,NaN,33-00,8.0
...,...,...,...,...,...,...,...,...,...
4519,68953,2024-07-19,19:00:00,avion 39,PDQ,2 ANOT: PARASOL CM1 Y ROLLER IZQ CM1,NT273,25-13,3.0
4520,68980,2024-07-20,06:25:00,avion 4,PDQ,MTTO SIN INFO,NT201,52-31,5.0
4521,69000,2024-07-20,14:27:00,avion 25,PIQ,SALTA AL ARRANCAR AVIONIC SINGLE SWITCH,NT141,42-11,30.0
4522,69007,2024-07-20,18:48:00,avion 25,PDQ,TEMPERATURA ALTA DE COMBUSTIBLE ENG #1,NT178,73-15,40.0


In [131]:
eventos_retrasos.info()
eventos_retrasos["ATA"]
eventos_retrasos.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4524 entries, 0 to 4523
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        4524 non-null   int64  
 1   Occurred  4524 non-null   object 
 2   At        4524 non-null   object 
 3   AC        4524 non-null   object 
 4   Station   4168 non-null   object 
 5   Headline  4524 non-null   object 
 6   FlightNo  3704 non-null   object 
 7   ATA       4524 non-null   object 
 8   Minutos   4517 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 318.2+ KB


ID            0
Occurred      0
At            0
AC            0
Station     356
Headline      0
FlightNo    820
ATA           0
Minutos       7
dtype: int64

#### Hay bastantes nulos en Station, FlightNo.
En el caso de Station vamos a aplicar la moda para que se identique las dos estaciones que más numero se repite.   
En lugar de eso y ser más realista, vamos a calcular el porcentaje que tiene cada estación y se le asignará la parte proporcional de los nulos.  
Los nulos de la columna FlightNo no influye que vuelo tuvo la avería por lo que haremos una secuencia de número consecutivos sin que haya duplicados

In [132]:
# Borrar las columnas "At" y "ID" de eventos_retrasos_ATA
eventos_retrasos.drop(columns=["At", "ID"], inplace=True)

In [133]:
eventos_retrasos

,Occurred,AC,Station,Headline,FlightNo,ATA,Minutos
0,2021-01-02,avion 1,PIQ,98254/1,NT421,77-00,38.0
1,2021-01-03,avion 2,PDQ,0097630/1,NT312,33-49,15.0
2,2021-01-04,avion 3,PIQ,MAINTENANCE ACTION WITHOUT PILOT REPORT - OPER...,414,25-65,3.0
3,2021-01-04,avion 3,PDQ,0096673/1,NT158,52-42,12.0
4,2021-01-05,avion 4,PIQ,OTHERS / NOT SPECIFIED - OPERATIONAL,NaN,33-00,8.0
...,...,...,...,...,...,...,...
4519,2024-07-19,avion 39,PDQ,2 ANOT: PARASOL CM1 Y ROLLER IZQ CM1,NT273,25-13,3.0
4520,2024-07-20,avion 4,PDQ,MTTO SIN INFO,NT201,52-31,5.0
4521,2024-07-20,avion 25,PIQ,SALTA AL ARRANCAR AVIONIC SINGLE SWITCH,NT141,42-11,30.0
4522,2024-07-20,avion 25,PDQ,TEMPERATURA ALTA DE COMBUSTIBLE ENG #1,NT178,73-15,40.0


In [134]:
nuevo_orden = ['Occurred','FlightNo','Station','AC','Minutos','ATA','Headline']
eventos_retrasos= eventos_retrasos[nuevo_orden]
eventos_retrasos

,Occurred,FlightNo,Station,AC,Minutos,ATA,Headline
0,2021-01-02,NT421,PIQ,avion 1,38.0,77-00,98254/1
1,2021-01-03,NT312,PDQ,avion 2,15.0,33-49,0097630/1
2,2021-01-04,414,PIQ,avion 3,3.0,25-65,MAINTENANCE ACTION WITHOUT PILOT REPORT - OPER...
3,2021-01-04,NT158,PDQ,avion 3,12.0,52-42,0096673/1
4,2021-01-05,NaN,PIQ,avion 4,8.0,33-00,OTHERS / NOT SPECIFIED - OPERATIONAL
...,...,...,...,...,...,...,...
4519,2024-07-19,NT273,PDQ,avion 39,3.0,25-13,2 ANOT: PARASOL CM1 Y ROLLER IZQ CM1
4520,2024-07-20,NT201,PDQ,avion 4,5.0,52-31,MTTO SIN INFO
4521,2024-07-20,NT141,PIQ,avion 25,30.0,42-11,SALTA AL ARRANCAR AVIONIC SINGLE SWITCH
4522,2024-07-20,NT178,PDQ,avion 25,40.0,73-15,TEMPERATURA ALTA DE COMBUSTIBLE ENG #1


In [135]:
# Calcular el porcentaje de cada estación distinta
station_counts = eventos_retrasos['Station'].value_counts(normalize=True) * 100

In [136]:
# Identificar el número de valores nulos en 'Station'
n_nulls_station = eventos_retrasos['Station'].isna().sum()

In [137]:
# Calcular la cantidad proporcional de nulos para cada estación
proportional_nulos = (station_counts / 100 * n_nulls_station).round().astype(int)

In [138]:
# Ajustar para asegurar que la suma total coincide con el número de nulos
difference = n_nulls_station - proportional_nulos.sum()

In [139]:
# Distribuir el desajuste a las estaciones más frecuentes
if difference > 0:
    proportional_nulos.iloc[:difference] += 1
elif difference < 0:
    proportional_nulos.iloc[:abs(difference)] -= 1

In [140]:
# Crear la lista ajustada de estaciones para rellenar los nulos
station_fill_values = np.concatenate([
    [station] * count for station, count in proportional_nulos.items()
])

In [141]:
# Rellenar los nulos en la columna "Station"
np.random.shuffle(station_fill_values)  # Mezclar para evitar sesgo por orden
eventos_retrasos.loc[eventos_retrasos['Station'].isna(), 'Station'] = station_fill_values

In [142]:
eventos_retrasos['Station'].isna().sum()

0

In [143]:
# Paso 1: Determinar el número total de filas nulas en "FlightNo"
num_nulos = eventos_retrasos['FlightNo'].isna().sum()
num_nulos

820

In [144]:
# Paso 2: Crear una lista de nuevos números consecutivos únicos
nuevos_numeros = [f"FL{num}" for num in range(1, num_nulos + 1)]

In [145]:
# Paso 3: Asignar los nuevos números a las filas nulas en "FlightNo"
eventos_retrasos.loc[eventos_retrasos['FlightNo'].isna(), 'FlightNo'] = nuevos_numeros

In [146]:
eventos_retrasos['FlightNo'].isna().sum()

0

In [147]:
# Calcular la media de la columna "Minutos"
mean_minutes = eventos_retrasos['Minutos'].mean()

# Usar loc para asignar valores de forma explícita
eventos_retrasos.loc[eventos_retrasos['Minutos'].isna(), 'Minutos'] = mean_minutes


In [148]:
eventos_retrasos['Minutos'].isna().sum()

0

In [149]:
# Verificar que no queden nulos
eventos_retrasos.isnull().sum()

Occurred    0
FlightNo    0
Station     0
AC          0
Minutos     0
ATA         0
Headline    0
dtype: int64

In [150]:
# Mostrar el resultado de la distribución
final_station_counts = eventos_retrasos['Station'].value_counts(normalize=True) * 100
final_station_counts

Station
PDQ    58.974359
PIQ    34.018568
HOR     2.166225
SMA     1.127321
TER     0.773652
CVU     0.596817
FNC     0.574713
MAD     0.375774
FLW     0.309461
GRW     0.221043
PMI     0.110522
ZAZ     0.088417
VGO     0.088417
OVD     0.088417
SDR     0.066313
XRY     0.044209
PXO     0.044209
RMU     0.044209
MRS     0.044209
PNA     0.044209
AVR     0.044209
SID     0.022104
GLN     0.022104
VIT     0.022104
AGA     0.022104
TLS     0.022104
RAK     0.022104
NKC     0.022104
Name: proportion, dtype: float64

In [151]:
eventos_retrasos["ATA"].value_counts()


ATA
05-20    210
12-11    158
UNK      157
25-00    116
52-31    115
        ... 
34-44      1
51-20      1
34-27      1
11-32      1
27-41      1
Name: count, Length: 421, dtype: int64

In [152]:
# Borrar columna ATA
eventos_retrasos.drop(columns=["Headline"], inplace=True, errors='ignore')

C:\Users\Victor\AppData\Local\Temp\ipykernel_11792\2997365217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventos_retrasos.drop(columns=["Headline"], inplace=True, errors='ignore')


In [153]:
eventos_retrasos

,Occurred,FlightNo,Station,AC,Minutos,ATA
0,2021-01-02,NT421,PIQ,avion 1,38.0,77-00
1,2021-01-03,NT312,PDQ,avion 2,15.0,33-49
2,2021-01-04,414,PIQ,avion 3,3.0,25-65
3,2021-01-04,NT158,PDQ,avion 3,12.0,52-42
4,2021-01-05,FL1,PIQ,avion 4,8.0,33-00
...,...,...,...,...,...,...
4519,2024-07-19,NT273,PDQ,avion 39,3.0,25-13
4520,2024-07-20,NT201,PDQ,avion 4,5.0,52-31
4521,2024-07-20,NT141,PIQ,avion 25,30.0,42-11
4522,2024-07-20,NT178,PDQ,avion 25,40.0,73-15


In [154]:
# Filtrar las filas utilizando comparación directa con cadenas de fecha y eliminar la hora de una vez
eventos_retrasos['Occurred'] = eventos_retrasos['Occurred'].str.slice(0, 10)

# Filtrar las filas dentro del rango de fechas
eventos_retrasos = eventos_retrasos[(eventos_retrasos['Occurred'] >= '2021-01-02') & (eventos_retrasos['Occurred'] <= '2023-09-30')]


C:\Users\Victor\AppData\Local\Temp\ipykernel_11792\3146643303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventos_retrasos['Occurred'] = eventos_retrasos['Occurred'].str.slice(0, 10)


In [155]:
eventos_retrasos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3313 entries, 0 to 3312
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Occurred  3313 non-null   object 
 1   FlightNo  3313 non-null   object 
 2   Station   3313 non-null   object 
 3   AC        3313 non-null   object 
 4   Minutos   3313 non-null   float64
 5   ATA       3313 non-null   object 
dtypes: float64(1), object(5)
memory usage: 181.2+ KB


In [156]:
# Crear un nuevo dataset eliminando el sub-ATA
eventos_retrasos["ATA_Principal"] = eventos_retrasos["ATA"].astype(str).str[:2]

C:\Users\Victor\AppData\Local\Temp\ipykernel_11792\2951716585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventos_retrasos["ATA_Principal"] = eventos_retrasos["ATA"].astype(str).str[:2]


In [157]:
# Verificar si la nueva columna se creó correctamente
print(eventos_retrasos[["ATA", "ATA_Principal"]].head())

     ATA ATA_Principal
0  77-00            77
1  33-49            33
2  25-65            25
3  52-42            52
4  33-00            33


In [158]:
# Revisar si hay valores faltantes en la nueva columna
print(eventos_retrasos["ATA_Principal"].isna().sum())

0


In [159]:
# Borrar columna ATA
eventos_retrasos.drop(columns=["ATA"], inplace=True, errors='ignore')

C:\Users\Victor\AppData\Local\Temp\ipykernel_11792\3785131746.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventos_retrasos.drop(columns=["ATA"], inplace=True, errors='ignore')


In [160]:
# Verificar
eventos_retrasos.head()

,Occurred,FlightNo,Station,AC,Minutos,ATA_Principal
0,2021-01-02,NT421,PIQ,avion 1,38.0,77
1,2021-01-03,NT312,PDQ,avion 2,15.0,33
2,2021-01-04,414,PIQ,avion 3,3.0,25
3,2021-01-04,NT158,PDQ,avion 3,12.0,52
4,2021-01-05,FL1,PIQ,avion 4,8.0,33


In [161]:
# Mostrar los valores únicos de 'ATA_Principal' ordenados
valores_unicos_ordenados = sorted(eventos_retrasos["ATA_Principal"].unique())
print(valores_unicos_ordenados)

['00', '05', '08', '09', '10', '11', '12', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '38', '42', '44', '45', '46', '47', '49', '50', '51', '52', '53', '54', '55', '56', '57', '61', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', 'UN']


In [162]:
# Cambiar el nombre de la columna 'ATA_Principal' a 'ATA'
eventos_retrasos.rename(columns={"ATA_Principal": "ATA"},inplace=True)

C:\Users\Victor\AppData\Local\Temp\ipykernel_11792\3225567720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventos_retrasos.rename(columns={"ATA_Principal": "ATA"},inplace=True)


In [163]:
eventos_retrasos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3313 entries, 0 to 3312
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Occurred  3313 non-null   object 
 1   FlightNo  3313 non-null   object 
 2   Station   3313 non-null   object 
 3   AC        3313 non-null   object 
 4   Minutos   3313 non-null   float64
 5   ATA       3313 non-null   object 
dtypes: float64(1), object(5)
memory usage: 181.2+ KB


In [164]:
eventos_retrasos.head()

,Occurred,FlightNo,Station,AC,Minutos,ATA
0,2021-01-02,NT421,PIQ,avion 1,38.0,77
1,2021-01-03,NT312,PDQ,avion 2,15.0,33
2,2021-01-04,414,PIQ,avion 3,3.0,25
3,2021-01-04,NT158,PDQ,avion 3,12.0,52
4,2021-01-05,FL1,PIQ,avion 4,8.0,33


In [165]:
# Cargar el archivo histmatriculas
histmatriculas = pd.read_csv("./data/histmatriculas.csv", encoding="latin1")

In [166]:
histmatriculas.head()

,Fecha,Numero vuelo,Aeropuerto Despegue Real,Aeropuerto Aterrizaje Real,Matricula Real,Flota Real
0,2021-01-01,105,PDQ,PIQ,AviÃ³n 18,ATR72
1,2021-01-01,106,PIQ,PDQ,AviÃ³n 18,ATR72
2,2021-01-01,110,PIQ,PDQ,AviÃ³n 25,ATR72
3,2021-01-01,128,CVU,PDQ,AviÃ³n 20,ATR72
4,2021-01-01,145,PDQ,PIQ,AviÃ³n 25,ATR72


In [167]:
histmatriculas["Matricula Real"] = "avion " + histmatriculas["Matricula Real"].str[6:]
histmatriculas

,Fecha,Numero vuelo,Aeropuerto Despegue Real,Aeropuerto Aterrizaje Real,Matricula Real,Flota Real
0,2021-01-01,105,PDQ,PIQ,avion 18,ATR72
1,2021-01-01,106,PIQ,PDQ,avion 18,ATR72
2,2021-01-01,110,PIQ,PDQ,avion 25,ATR72
3,2021-01-01,128,CVU,PDQ,avion 20,ATR72
4,2021-01-01,145,PDQ,PIQ,avion 25,ATR72
...,...,...,...,...,...,...
199985,2023-09-30,933,PXO,FNC,avion 40,ATR72
199986,2023-09-30,9400,PDQ,VCE,avion 32,EM295
199987,2023-09-30,9401,VCE,PDQ,avion 32,EM295
199988,2023-09-30,9600,PDQ,FLR,avion 12,EM295


In [168]:
# Borramos columnas que no necesitamos en ambos datasets

# Borrar la columna "Aeropuerto Aterrizaje Real" de histmatriculas
histmatriculas.drop(columns=["Aeropuerto Aterrizaje Real","Flota Real"], inplace=True)

In [169]:
# Buscamos si hay duplicados en ambos datasets
duplicados = histmatriculas[histmatriculas.duplicated()]
len(duplicados)

3

In [170]:
eventos_duplicados = eventos_retrasos[eventos_retrasos.duplicated()]
len(eventos_duplicados)

0

In [171]:
# Eliminamos duplicados en histmatriculas
histmatriculas.drop_duplicates(inplace=True)
len(histmatriculas)

199987

In [172]:
# Combinamos ambos datasets usando sqlite3

conector = sqlite3.connect("base_de_datos.db")
query = conector.cursor()

# Renombrar las columnas en histmatriculas para que coincidan con eventos_retrasos
histmatriculas.rename(columns={
    'Numero vuelo': 'Numero_vuelo',
    'Aeropuerto Despegue Real': 'Station',  # Equivale a "Station" en eventos_retrasos
    'Matricula Real': 'AC'  # Equivale a "AC" en eventos_retrasos
}, inplace=True)

# Añadir columnas "Minutos" y "ATA" a histmatriculas
histmatriculas['Minutos'] = 0.0
histmatriculas['ATA'] = '99'

In [173]:
# Renombrar las columnas en eventos_retrasos para que coincidan con histmatriculas
eventos_retrasos.rename(columns={
    'Occurred': 'Fecha',  # Equivale a "Fecha" en histmatriculas
    'FlightNo': 'Numero_vuelo'  # Equivale a "Numero_vuelo" en histmatriculas
}, inplace=True)

C:\Users\Victor\AppData\Local\Temp\ipykernel_11792\555987341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eventos_retrasos.rename(columns={


In [174]:
eventos_retrasos

,Fecha,Numero_vuelo,Station,AC,Minutos,ATA
0,2021-01-02,NT421,PIQ,avion 1,38.0,77
1,2021-01-03,NT312,PDQ,avion 2,15.0,33
2,2021-01-04,414,PIQ,avion 3,3.0,25
3,2021-01-04,NT158,PDQ,avion 3,12.0,52
4,2021-01-05,FL1,PIQ,avion 4,8.0,33
...,...,...,...,...,...,...
3308,2023-09-30,NT309,PDQ,avion 23,18.0,29
3309,2023-09-30,NT420,PIQ,avion 16,41.0,21
3310,2023-09-30,NT5251,PIQ,avion 15,58.0,25
3311,2023-09-30,NT147,PDQ,avion 14,56.0,24


In [175]:
histmatriculas

,Fecha,Numero_vuelo,Station,AC,Minutos,ATA
0,2021-01-01,105,PDQ,avion 18,0.0,99
1,2021-01-01,106,PIQ,avion 18,0.0,99
2,2021-01-01,110,PIQ,avion 25,0.0,99
3,2021-01-01,128,CVU,avion 20,0.0,99
4,2021-01-01,145,PDQ,avion 25,0.0,99
...,...,...,...,...,...,...
199985,2023-09-30,933,PXO,avion 40,0.0,99
199986,2023-09-30,9400,PDQ,avion 32,0.0,99
199987,2023-09-30,9401,VCE,avion 32,0.0,99
199988,2023-09-30,9600,PDQ,avion 12,0.0,99


In [176]:
# Identificar columnas comunes entre las dos tablas
columnas_comunes = histmatriculas.columns.intersection(eventos_retrasos.columns)

In [177]:
# Filtrar ambas tablas para mantener solo las columnas comunes y ordenar por las mismas columnas
histmatriculas = histmatriculas[columnas_comunes]
eventos_retrasos = eventos_retrasos[columnas_comunes]

In [178]:
# Tabla para histmatriculas
query.execute('''
    CREATE TABLE IF NOT EXISTS histmatriculas (
        Fecha TEXT,
        Numero_vuelo TEXT,
        Station TEXT,
        AC TEXT,
        Minutos REAL,
        ATA TEXT
    )
''')

# Tabla para eventos_retrasos
query.execute('''
    CREATE TABLE IF NOT EXISTS eventos_retrasos (
         Fecha TEXT,
        Numero_vuelo TEXT,
        Station TEXT,
        AC TEXT,
        Minutos REAL,
        ATA TEXT      
    )
''')


# Insertar los datos en las tablas
histmatriculas.to_sql("histmatriculas", conector, if_exists="replace", index=False)
eventos_retrasos.to_sql("eventos_retrasos", conector, if_exists="replace", index=False)


In [179]:
# Verificar que "Flota" no está en la tabla SQLite
query.execute("PRAGMA table_info(histmatriculas)")
print(query.fetchall())

[(0, 'Fecha', 'TEXT', 0, None, 0), (1, 'Numero_vuelo', 'TEXT', 0, None, 0), (2, 'Station', 'TEXT', 0, None, 0), (3, 'AC', 'TEXT', 0, None, 0), (4, 'Minutos', 'REAL', 0, None, 0), (5, 'ATA', 'TEXT', 0, None, 0)]


query.execute('''
    CREATE TABLE combinada AS
    SELECT Fecha, Numero_vuelo, Station, AC, Minutos, ATA
    FROM histmatriculas
    UNION ALL
    SELECT Fecha, Numero_vuelo, Station, AC, Minutos, ATA
    FROM eventos_retrasos;
''')

In [180]:
# Eliminar la tabla combinada si ya existe
query.execute("DROP TABLE IF EXISTS combinada")

In [181]:
# Crear la tabla combinada con las columnas actualizadas
query.execute('''
    CREATE TABLE combinada AS
    SELECT Fecha, Numero_vuelo, Station, AC , Minutos, ATA
    FROM histmatriculas
    UNION ALL
    SELECT Fecha, Numero_vuelo, Station, AC, Minutos, ATA
    FROM eventos_retrasos
''')

# Crear una tabla combinada que une ambos datasets
query.execute('''
    CREATE TABLE IF NOT EXISTS combinada AS
    SELECT * FROM histmatriculas
    UNION ALL
    SELECT * FROM eventos_retrasos
''')


In [182]:
# Leer la tabla combinada desde la base de datos y mostrar algunas filas
query.execute('SELECT * FROM combinada LIMIT 5')
resultados = query.fetchall()
for fila in resultados:
    print(fila)



('2021-01-01', '105', 'PDQ', 'avion  18', 0.0, '99')
('2021-01-01', '106', 'PIQ', 'avion  18', 0.0, '99')
('2021-01-01', '110', 'PIQ', 'avion  25', 0.0, '99')
('2021-01-01', '128', 'CVU', 'avion  20', 0.0, '99')
('2021-01-01', '145', 'PDQ', 'avion  25', 0.0, '99')


In [183]:
# Consultar las filas de la tabla combinada
query.execute("SELECT * FROM combinada")
filas = query.fetchall()

# Guardar como archivo CSV en la subcarpeta "data"
with open( "./data/combinada.csv", "w", newline="", encoding="latin1") as f:
    writer = csv.writer(f)
    writer.writerow([description[0] for description in query.description])  # Escribir la cabecera del CSV
    writer.writerows(filas)  # Escribir las filas

# Guardar los cambios en la base de datos
conector.commit()

# Cerrar la conexión
query.close()
conector.close()

print("Archivo guardado en la subcarpeta 'data' como 'combinada.csv'.")

Archivo guardado en la subcarpeta 'data' como 'combinada.csv'.


Para construir un modelo de predicción de averías, **es importante tener en cuenta todos los vuelos realizados**. Aquí te explico por qué:

### Razones para incluir todos los vuelos:
1. **Evitar sesgo de selección**:
   - Si solo incluyes vuelos con averías, el modelo aprenderá únicamente sobre vuelos con problemas y no tendrá contexto sobre los vuelos sin problemas. Esto podría llevar a un modelo incapaz de diferenciar entre condiciones normales y condiciones de fallo.

2. **Balance de clases**:
   - La mayoría de los vuelos suelen ser exitosos, y las averías representan un subconjunto pequeño. Incluir todos los vuelos permite capturar este desequilibrio, lo que es esencial para crear un modelo realista que pueda predecir averías infrecuentes.

3. **Factores no relacionados con averías**:
   - Algunas características (como el tipo de avión, ruta, condiciones climáticas) podrían ser relevantes para predecir tanto vuelos exitosos como fallidos. Si excluyes vuelos sin averías, pierdes esta información.

4. **Mejor evaluación del modelo**:
   - Al incluir todos los vuelos, puedes calcular métricas como precisión, sensibilidad y especificidad para evaluar cómo el modelo equilibra la predicción entre vuelos con y sin averías.

### Cómo tratar el desequilibrio:
Si las averías son mucho menos comunes que los vuelos sin problemas, podrías necesitar técnicas para manejar este desequilibrio, como:
- **Submuestreo**: Reducir el número de vuelos sin averías en el conjunto de entrenamiento.
- **Sobremuestreo**: Aumentar el número de vuelos con averías (duplicando datos o utilizando técnicas como SMOTE).
- **Ajuste de pesos**: Penalizar errores en vuelos con averías más fuertemente en la función de pérdida.

Incluir todos los vuelos es fundamental para un análisis más completo y un modelo más robusto. Si quieres, puedo ayudarte a preparar el conjunto de datos para esta tarea.